# Q&A about the document

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/QA_for_document.ipynb.ipynb)

In [3]:
%%capture
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install tiktoken

In [8]:
from langchain.chains.question_answering import load_qa_chain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.llms import OpenAI

In [13]:
file_path = "/home/jovyan/langchain/data/sample.txt"


#  データの用意
with open(file_path) as f:
    dragonball_txt = f.read()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_text(dragonball_txt)


query = "Kaggle社内Meet upとはどんなものですか?"

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)
docs = docsearch.similarity_search(query)

In [10]:
llm = OpenAI(temperature=0, model_name="text-davinci-002")

### Stuff
* 全ての関連データをコンテキストとしてプロンプトに詰め込み，言語モデルに渡す手法
    * メリット：LLMへの呼び出しは一回のみになり，テキスト生成時にLLMは一度に全てのデータを参照できる
    * デメリット：LLMのコンテキストの長さによる制限により大きなデータでは機能しない

In [11]:
chain = load_qa_chain(llm, chain_type="stuff")

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Kaggle社内Meet upは、Kaggleに参加したことがある社員や参加したことはないけれど興味がある社員、Kaggleに限らず技術コンペについて情報収集したい社員が、交流できるように企画されたイベントです。'}

### Map Reduce
* 関連データをチャンクに分割し，チャンクごとにプロンプトを作り言語モデルに渡す．その後，それらの結果を結合するプロンプトを言語モデルに渡す
    * メリット：Stuffingより大きなデータが扱える．チャンクのLLMの呼び出しを並列実行できる．
    * デメリット：Stuffingより多くの回数のLLMの呼び出しが必要になる．また最後の結合により一部のデータを失う

In [14]:
chain = load_qa_chain(llm, chain_type="map_reduce")

chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': "\n\nI don't know."}

In [ ]:
chain = load_qa_chain(llm, chain_type="refine")

chain({"input_documents": docs, "question": query}, return_only_outputs=True)